In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from os import listdir
from os.path import join
from tqdm import tnrange


%matplotlib inline
i_path='raw/mpd/data'

In [2]:

slices=listdir(i_path)
tracks_uri=set([''])
artists_uri=set([''])
albums_uri=set([''])
tracks=[]
artists=[]
albums=[]
playlists=[]
playlist_track_rel=[]
album_track_rel=[]
artist_track_rel=[]

for i in tnrange(len(slices)):
    s=slices[i]
    with open(join(i_path,s)) as f:
        plist=json.load(f)
        for p in plist['playlists']:
            for t in p['tracks']:
                if t['track_uri'] not in tracks_uri:
                    tracks_uri.add(t['track_uri'])
                    
                    tr={}
                    tr['track_uri']=t['track_uri']
                    tr['track_name']=t['track_name']
                    tr['duration_ms']=t['duration_ms']
                    tracks.append(tr)
                    
                if t['album_uri'] not in albums_uri:
                    albums_uri.add(t['album_uri'])
                    
                    al={}
                    al['album_uri']=t['album_uri']
                    al['album_name']=t['album_name']
                    albums.append(al)
                    
                if t['artist_uri'] not in artists_uri:
                    artists_uri.add(t['artist_uri'])
                    
                    ar={}
                    ar['artist_uri']=t['artist_uri']
                    ar['artist_name']=t['artist_name']
                    artists.append(ar)

tracks_uri.remove('')
albums_uri.remove('')
artists_uri.remove('')

tracks_id={t:i for i,t in enumerate(sorted(list(tracks_uri)))}
albums_id={t:i for i,t in enumerate(sorted(list(albums_uri)))}
artists_id={t:i for i,t in enumerate(sorted(list(artists_uri)))}

for i in range(len(tracks)):
    tracks[i]['track_id']=tracks_id[tracks[i]['track_uri']]
    del tracks[i]['track_uri']
    
for i in range(len(albums)):
    albums[i]['album_id']=albums_id[albums[i]['album_uri']]
    del albums[i]['album_uri']
    
for i in range(len(artists)):
    artists[i]['artist_id']=artists_id[artists[i]['artist_uri']]
    del artists[i]['artist_uri']
    
tracks_data=pd.DataFrame(tracks)
albums_data=pd.DataFrame(albums)
artists_data=pd.DataFrame(artists)
                

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [4]:
al_tr=set([''])
ar_tr=set([''])
for i in tnrange(len(slices)):
    s=slices[i]
    with open(join(i_path,s)) as f:
        sl=json.load(f)
        
        for pl in sl['playlists']:
            tracks_i = pl['tracks']
            del pl['tracks']
            playlists.append(pl)
            
            for tr in tracks_i:
                
                pt={}
                pt['pid']=pl['pid']
                pt['track_id']=tracks_id[tr['track_uri']]
                pt['pos']=tr['pos']
                playlist_track_rel.append(pt)
                
                at={}
                at['album_id']=albums_id[tr['album_uri']]
                at['track_id']=tracks_id[tr['track_uri']]
                if (at['album_id'],at['track_id']) not in al_tr:
                    al_tr.add((at['album_id'],at['track_id']))
                    album_track_rel.append(at)
                
                at={}
                at['artist_id']=artists_id[tr['artist_uri']]
                at['track_id']=tracks_id[tr['track_uri']]
                if (at['artist_id'],at['track_id']) not in ar_tr:
                    ar_tr.add((at['artist_id'],at['track_id']))
                    artist_track_rel.append(at)
                
playlist_data=pd.DataFrame(playlists)
playlist_track_data=pd.DataFrame(playlist_track_rel)
album_track_data=pd.DataFrame(album_track_rel)
artist_track_data=pd.DataFrame(artist_track_rel)

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [11]:
with open('raw/tracks_uri_2_id.json','w') as f:
    json.dump(tracks_id,f)

In [19]:
playlist_track_data.to_csv('raw/large/playlist_tracks.csv',index=False)

In [20]:
tracks_data.to_csv('raw/large/tracks.csv',index=False)
albums_data.to_csv('raw/large/albums.csv',index=False)
artists_data.to_csv('raw/large/artists.csv',index=False)
playlist_data.to_csv('raw/large/playlists.csv',index=False)
album_track_data.to_csv('raw/large/album_tracks.csv',index=False)
artist_track_data.to_csv('raw/large/artist_tracks.csv',index=False)

In [23]:
small_playlists=playlist_data[:10000]
playlist_ids=set(small_playlists.pid)
small_playlist_track=playlist_track_data[playlist_track_data.pid.apply(lambda p:p in playlist_ids)]
track_ids=set(small_playlist_track.track_id)
small_tracks=tracks_data[tracks_data.track_id.apply(lambda t:t in track_ids)]
small_album_track=album_track_data[album_track_data.track_id.apply(lambda t:t in track_ids)]
small_artist_track=artist_track_data[artist_track_data.track_id.apply(lambda t:t in track_ids)]
album_ids=set(small_album_track.album_id)
small_albums=albums_data[albums_data.album_id.apply(lambda a:a in album_ids)]
artist_ids=set(small_artist_track.artist_id)
small_artists=artists_data[artists_data.artist_id.apply(lambda a:a in artist_ids)]

In [24]:
small_tracks.to_csv('raw/small/tracks.csv',index=False)
small_albums.to_csv('raw/small/albums.csv',index=False)
small_artists.to_csv('raw/small/artists.csv',index=False)
small_playlists.to_csv('raw/small/playlists.csv',index=False)
small_album_track.to_csv('raw/small/album_tracks.csv',index=False)
small_artist_track.to_csv('raw/small/artist_tracks.csv',index=False)
small_playlist_track.to_csv('raw/small/playlist_tracks.csv',index=False)

In [25]:
playlists=[]
playlist_track_rel=[]
with open('raw/challenge/challenge_set.json') as f:
    sl=json.load(f)

    for pl in sl['playlists']:
        tracks_i = pl['tracks']
        del pl['tracks']
        playlists.append(pl)

        for tr in tracks_i:

            pt={}
            pt['pid']=pl['pid']
            pt['track_id']=tracks_id[tr['track_uri']]
            pt['pos']=tr['pos']
            playlist_track_rel.append(pt)

playlist_test=pd.DataFrame(playlists)
playlist_track_test=pd.DataFrame(playlist_track_rel)


In [27]:
playlist_track_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281000 entries, 0 to 280999
Data columns (total 3 columns):
pid         281000 non-null int64
pos         281000 non-null int64
track_id    281000 non-null int64
dtypes: int64(3)
memory usage: 6.4 MB


In [29]:
playlist_test.to_csv('raw/test/playlists.csv',index=False)
playlist_track_test.to_csv('raw/test/playlist_tracks.csv',index=False)